## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-13-06-55-41 +0000,bbc,Badenoch demands PM address 'unanswered' China...,https://www.bbc.com/news/articles/cg424d712q7o...
1,2025-10-13-06-55-00 +0000,wsj,What to know about the Gaza deal between Israe...,https://www.wsj.com/world/middle-east/israel-h...
2,2025-10-13-06-54-00 +0000,wsj,Lloyds Banking Group Sets Aside Further $1.07 ...,https://www.wsj.com/finance/banking/lloyds-ban...
3,2025-10-13-06-53-39 +0000,bbc,Reeves urged to avoid 'half-baked' tax fixes i...,https://www.bbc.com/news/articles/cx2n08n15w2o...
4,2025-10-13-06-47-51 +0000,nyt,What We Know About the Deal Between Israel and...,https://www.nytimes.com/2025/10/09/world/middl...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2352/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
37,trump,28
5,china,18
12,israel,15
13,hamas,14
36,hostages,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
80,2025-10-12-22-52-05 +0000,nypost,Hillary Clinton heaps rare praise on President...,https://nypost.com/2025/10/12/us-news/hillary-...,91
163,2025-10-12-12-36-07 +0000,nypost,Hamas ready to begin freeing 20 living Israeli...,https://nypost.com/2025/10/12/world-news/hamas...,90
58,2025-10-13-00-41-23 +0000,bbc,Trump says 'war is over' in Gaza as he flies t...,https://www.bbc.com/news/articles/cn409y125v3o...,84
18,2025-10-13-05-31-15 +0000,nypost,‘War is over’ Trump declares on Air Force One ...,https://nypost.com/2025/10/13/world-news/war-i...,81
5,2025-10-13-06-47-09 +0000,wapo,Live updates: Hamas releases first 7 hostages ...,https://www.washingtonpost.com/world/2025/10/1...,74


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
80,91,2025-10-12-22-52-05 +0000,nypost,Hillary Clinton heaps rare praise on President...,https://nypost.com/2025/10/12/us-news/hillary-...
207,51,2025-10-12-07-22-45 +0000,nypost,"Chinese coast guard fires water cannon, rams P...",https://nypost.com/2025/10/12/world-news/chine...
82,45,2025-10-12-22-22-15 +0000,nypost,Israeli hostage release feels tantalizingly cl...,https://nypost.com/2025/10/12/us-news/hundreds...
58,35,2025-10-13-00-41-23 +0000,bbc,Trump says 'war is over' in Gaza as he flies t...,https://www.bbc.com/news/articles/cn409y125v3o...
122,29,2025-10-12-18-39-55 +0000,nypost,Kamala Harris reveals Biden sent her call to v...,https://nypost.com/2025/10/12/us-news/kamala-h...
45,27,2025-10-13-01-39-12 +0000,nypost,"Elderly, dementia-stricken veteran found dead ...",https://nypost.com/2025/10/12/us-news/elderly-...
189,27,2025-10-12-09-30-00 +0000,wsj,Hyundai Factory Was a Deadly Job Site Before I...,https://www.wsj.com/us-news/hyundai-factory-wa...
54,26,2025-10-13-01-03-00 +0000,wsj,Answers to frequently asked questions about Th...,https://www.wsj.com/economy/economic-forecasti...
39,25,2025-10-13-03-13-10 +0000,nypost,Wild video shows Texas airport worker lose con...,https://nypost.com/2025/10/12/us-news/wild-vid...
60,24,2025-10-13-00-30-54 +0000,nyt,Border Clash Between Afghanistan and Pakistan ...,https://www.nytimes.com/2025/10/12/world/asia/...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
